# LangChain: 모델, 프롬프트 및 출력 파서


## 개요

 * OpenAI에 대한 직접 API 호출 
 * LangChain을 통한 API 호출: 
     * 프롬프트 
     * 모델 
     * 출력 파서


In [133]:
# !pip install python-dotenv
# !pip install openai

In [2]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

## 채팅 API : OpenAI

OpenAI에 직접 API를 호출하는 것부터 시작하겠습니다.

In [3]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, 
    )
    return response.choices[0].message["content"]


In [4]:
get_completion("What is 1+1?")

'As an AI language model, I can tell you that the answer to 1+1 is 2.'

In [11]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse,\
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

In [12]:
style = """American English \
in a calm and respectful tone
"""

In [13]:
prompt = f"""Translate the text \
that is delimited by triple backticks 
into a style that is {style}.
text: ```{customer_email}```
"""

print(prompt)

Translate the text that is delimited by triple backticks 
into a style that is American English in a calm and respectful tone
.
text: ```
Arrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse,the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!
```



In [14]:
response = get_completion(prompt)

In [15]:
response

'I am quite upset that my blender lid came off and caused my smoothie to splatter all over my kitchen walls. Additionally, the warranty does not cover the cost of cleaning up the mess. Would you be able to assist me, please? Thank you kindly.'

In [16]:
customer_email = """
아, 블렌더 뚜껑이 날아가서 스무디가 주방 벽에 튀어서 화가 났어요! 
설상가상으로, 보증은 주방 청소 비용을 포함하지 않습니다. 
지금 당장 도움이 필요해요!
"""

In [17]:
style = """
한국어 \ 차분하고 정중한 어조로
"""

In [18]:
prompt = f"""Translate the text \
that is delimited by triple backticks 
into a style that is {style}.
text: ```{customer_email}```
"""

print(prompt)

Translate the text that is delimited by triple backticks 
into a style that is 
한국어 \ 차분하고 정중한 어조로
.
text: ```
아, 블렌더 뚜껑이 날아가서 스무디가 주방 벽에 튀어서 화가 났어요! 
설상가상으로, 보증은 주방 청소 비용을 포함하지 않습니다. 
지금 당장 도움이 필요해요!
```



In [19]:
response = get_completion(prompt)

In [20]:
response

'아, 블렌더 뚜껑이 날아가서 스무디가 주방 벽에 튀어서 화가 났어요. \n게다가, 보증은 주방 청소 비용을 포함하지 않습니다. \n지금 당장 도움이 필요합니다.'

## 채팅 API : LangChain

LangChain을 사용하여 동일한 작업을 수행하는 방법을 시도해 보겠습니다.

In [23]:
#!pip install --upgrade langchain

### Model

In [24]:
from langchain.chat_models import ChatOpenAI

In [25]:
# LLM으로 생성된 텍스트의 무작위성과 창의성을 제어하려면 온도 = 0.0을 사용합니다.

chat = ChatOpenAI(temperature=0.0)
chat

ChatOpenAI(verbose=False, callbacks=None, callback_manager=None, client=<class 'openai.api_resources.chat_completion.ChatCompletion'>, model_name='gpt-3.5-turbo', temperature=0.0, model_kwargs={}, openai_api_key=None, openai_api_base=None, openai_organization=None, openai_proxy=None, request_timeout=None, max_retries=6, streaming=False, n=1, max_tokens=None)

### Prompt template

In [26]:
template_string = """Translate the text \
that is delimited by triple backticks \
into a style that is {style}. \
text: ```{text}```
"""

In [27]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(template_string)


In [28]:
prompt_template.messages[0].prompt

PromptTemplate(input_variables=['style', 'text'], output_parser=None, partial_variables={}, template='Translate the text that is delimited by triple backticks into a style that is {style}. text: ```{text}```\n', template_format='f-string', validate_template=True)

In [29]:
prompt_template.messages[0].prompt.input_variables

['style', 'text']

In [30]:
customer_style = """American English \
in a calm and respectful tone
"""

In [31]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse, \
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

In [32]:
customer_messages = prompt_template.format_messages(
                    style=customer_style,
                    text=customer_email)

In [33]:
print(type(customer_messages))
print(type(customer_messages[0]))

<class 'list'>
<class 'langchain.schema.HumanMessage'>


In [34]:
print(customer_messages[0])

content="Translate the text that is delimited by triple backticks into a style that is American English in a calm and respectful tone\n. text: ```\nArrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse, the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!\n```\n" additional_kwargs={} example=False


In [37]:
# 고객 메시지의 스타일을 번역하려면 LLM을 호출합니다.
customer_response = chat(customer_messages)

In [38]:
print(customer_response.content)

I'm really frustrated that my blender lid flew off and made a mess of my kitchen walls with smoothie. To add to my frustration, the warranty doesn't cover the cost of cleaning up my kitchen. Can you please help me out, friend?


In [39]:
customer_style = """Korean \
in a calm and respectful tone
"""

In [40]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse, \
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

In [41]:
customer_messages = prompt_template.format_messages(
                    style=customer_style,
                    text=customer_email)

In [42]:
# 고객 메시지의 스타일을 번역하려면 LLM을 호출합니다.
customer_response = chat(customer_messages)

In [43]:
print(customer_response.content)

저는 제 믹서기 뚜껑이 날아가서 스무디가 주방 벽에 튀어서 화가 나요. 게다가 보증이 청소 비용을 보장하지 않아 더욱 안타깝습니다. 지금 당장 도와주세요, 친구야.


In [44]:
service_reply = """Hey there customer, \
the warranty does not cover \
cleaning expenses for your kitchen \
because it's your fault that \
you misused your blender \
by forgetting to put the lid on before \
starting the blender. \
Tough luck! See ya!
"""

In [45]:
service_style_pirate = """\
a polite tone \
that speaks in English Pirate\
"""

In [46]:
service_messages = prompt_template.format_messages(
    style=service_style_pirate,
    text=service_reply)

print(service_messages[0].content)

Translate the text that is delimited by triple backticks into a style that is a polite tone that speaks in English Pirate. text: ```Hey there customer, the warranty does not cover cleaning expenses for your kitchen because it's your fault that you misused your blender by forgetting to put the lid on before starting the blender. Tough luck! See ya!
```



In [47]:
service_response = chat(service_messages)
print(service_response.content)

Ahoy there, me hearty customer! I be sorry to inform ye that the warranty be not coverin' the expenses o' cleaning yer galley, as it be yer own fault fer misusin' yer blender by forgettin' to put the lid on afore startin' it. Aye, tough luck! Farewell and may the winds be in yer favor!


In [48]:
service_reply = """Hey there customer, \
the warranty does not cover \
cleaning expenses for your kitchen \
because it's your fault that \
you misused your blender \
by forgetting to put the lid on before \
starting the blender. \
Tough luck! See ya!
"""

In [49]:
service_style_pirate = """\
a polite tone \
that speaks in Korean\
"""

In [50]:
service_messages = prompt_template.format_messages(
    style=service_style_pirate,
    text=service_reply)

print(service_messages[0].content)

Translate the text that is delimited by triple backticks into a style that is a polite tone that speaks in Korean. text: ```Hey there customer, the warranty does not cover cleaning expenses for your kitchen because it's your fault that you misused your blender by forgetting to put the lid on before starting the blender. Tough luck! See ya!
```



In [51]:
service_response = chat(service_messages)
print(service_response.content)

고객님, 안녕하세요. 저희 보증은 부주의로 인한 블렌더 사용 오류로 인한 청소 비용은 보상하지 않습니다. 뚜렷한 사용 설명서를 참고하시어 블렌더 사용 전에 뚜껑을 꼭 닫으시기 바랍니다. 불편을 드려 죄송합니다. 좋은 하루 보내세요.


## Output Parsers

LLM 출력의 모양을 정의하는 것부터 시작해 보겠습니다:

In [52]:
{
  "gift": False,
  "delivery_days": 5,
  "price_value": "pretty affordable!"
}

{'gift': False, 'delivery_days': 5, 'price_value': 'pretty affordable!'}

In [53]:
customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

review_template = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product \
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: {text}
"""

In [67]:
customer_review1 = """\
이 나뭇잎 송풍기는 꽤 놀랍습니다. 촛불, 산들바람, 바람이 부는 도시, 토네이도의 네 가지 설정이 있습니다. \
아내의 결혼 기념일 선물에 맞춰 이틀 만에 도착했습니다. \
아내가 너무 좋아해서 할 말을 잃을 정도였어요. \
지금까지는 저 혼자만 사용하고 있으며, 격일로 아침마다 잔디밭의 낙엽을 치우는 데 사용하고 있습니다. \
다른 잎사귀 송풍기보다 약간 비싸지만 추가 기능을 고려하면 그만한 가치가 있다고 생각해요."""

review_template1 = """\ 다음 텍스트의 경우 다음 정보를 추출합니다:

gift: 다른 사람을 위해 선물용으로 구매하셨나요? \
예인 경우 True, 아니오 또는 알 수 없는 경우 False로 응답합니다.

delivery_days: 제품이 도착하는 데 \
며칠이 걸렸나요? 이 정보를 찾을 수 없으면 -1을 출력합니다.

price_value: 값 또는 가격에 대한 모든 문장을 추출하여 쉼표로 구분된 Python 목록으로 출력합니다.

다음 키를 사용하여 출력을 JSON으로 형식화합니다. 
gift
delivery_days
price_value

text: {text} 
"""

In [68]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(review_template)
print(prompt_template)

input_variables=['text'] output_parser=None partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], output_parser=None, partial_variables={}, template='For the following text, extract the following information:\n\ngift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.\n\ndelivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.\n\nprice_value: Extract any sentences about the value or price,and output them as a comma separated Python list.\n\nFormat the output as JSON with the following keys:\ngift\ndelivery_days\nprice_value\n\ntext: {text}\n', template_format='f-string', validate_template=True), additional_kwargs={})]


In [69]:
prompt_template1 = ChatPromptTemplate.from_template(review_template1)
print(prompt_template1)

input_variables=['text'] output_parser=None partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], output_parser=None, partial_variables={}, template='\\ 다음 텍스트의 경우 다음 정보를 추출합니다:\n\ngift: 다른 사람을 위해 선물용으로 구매하셨나요? 예인 경우 True, 아니오 또는 알 수 없는 경우 False로 응답합니다.\n\ndelivery_days: 제품이 도착하는 데 며칠이 걸렸나요? 이 정보를 찾을 수 없으면 -1을 출력합니다.\n\nprice_value: 값 또는 가격에 대한 모든 문장을 추출하여 쉼표로 구분된 Python 목록으로 출력합니다.\n\n다음 키를 사용하여 출력을 JSON으로 형식화합니다. \ngift\ndelivery_days\nprice_value\n\ntext: {text} \n', template_format='f-string', validate_template=True), additional_kwargs={})]


In [70]:
messages = prompt_template.format_messages(text=customer_review)
chat = ChatOpenAI(temperature=0.0)
response = chat(messages)
print(response.content)


{
    "gift": true,
    "delivery_days": 2,
    "price_value": ["It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."]
}


In [73]:
messages1 = prompt_template1.format_messages(text=customer_review1)
chat1 = ChatOpenAI(temperature=0.0)
response1 = chat(messages1)
print(response1.content)


{
  "gift": true,
  "delivery_days": 2,
  "price_value": ["추가 기능을 고려하면 그만한 가치가 있다고 생각해요."]
}


In [71]:
type(response.content)

str

In [76]:
type(response1.content)

str

In [77]:
# 'gift'는 사전이 아니기 때문에 
# 이 코드 줄을 실행하면 오류가 발생합니다 
# 'gift'는 문자열입니다.
response.content.get('gift')

AttributeError: 'str' object has no attribute 'get'

### LLM 출력 문자열을 파이썬 dictionary로 파싱합니다.

In [78]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

In [113]:
# gift_schema = ResponseSchema(name="gift",
#                              description="Was the item purchased\
#                              as a gift for someone else? \
#                              Answer True if yes,\
#                              False if not or unknown.")
# delivery_days_schema = ResponseSchema(name="delivery_days",
#                                       description="How many days\
#                                       did it take for the product\
#                                       to arrive? If this \
#                                       information is not found,\
#                                       output -1.")
# price_value_schema = ResponseSchema(name="price_value",
#                                     description="Extract any\
#                                     sentences about the value or \
#                                     price, and output them as a \
#                                     comma separated Python list.")

gift_schema = ResponseSchema(name="gift", description="gift")
delivery_days_schema = ResponseSchema(name="delivery_days", description="delivery days")
price_value_schema = ResponseSchema(name="price_value", description="price value")

response_schemas = [gift_schema, 
                    delivery_days_schema,
                    price_value_schema]

In [114]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [115]:
format_instructions = output_parser.get_format_instructions()

In [116]:
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "\`\`\`json" and "\`\`\`":

```json
{
	"gift": string  // gift
	"delivery_days": string  // delivery days
	"price_value": string  // price value
}
```


In [117]:
review_template_2 = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product\
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

text: {text}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template=review_template_2)

messages = prompt.format_messages(text=customer_review, 
                                format_instructions=format_instructions)

In [118]:
print(messages[0].content)

For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the productto arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,and output them as a comma separated Python list.

text: This leaf blower is pretty amazing.  It has four settings:candle blower, gentle breeze, windy city, and tornado. It arrived in two days, just in time for my wife's anniversary present. I think my wife liked it so much she was speechless. So far I've been the only one using it, and I've been using it every other morning to clear the leaves on our lawn. It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features.


The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "\`\`\`json" and "

In [119]:
response = chat(messages)

In [120]:
print(response.content)

```json
{
	"gift": true,
	"delivery_days": "2",
	"price_value": ["It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."]
}
```


In [121]:
output_dict = output_parser.parse(response.content)

In [122]:
output_dict

{'gift': True,
 'delivery_days': '2',
 'price_value': ["It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."]}

In [123]:
type(output_dict)

dict

In [124]:
output_dict.get('delivery_days')

'2'

In [125]:
review_template_3 = """\
다음 텍스트의 경우 다음 정보를 추출합니다:

gift: 다른 사람을 위해 선물용으로 구매하셨나요? \
예인 경우 True, 아니오 또는 알 수 없는 경우 False로 응답합니다.

delivery_days: 제품이 도착하는 데 \
며칠이 걸렸나요? 이 정보를 찾을 수 없으면 -1을 출력합니다.

price_value: 값 또는 가격에 대한 모든 문장을 추출하여 쉼표로 구분된 Python 목록으로 출력합니다.

text: {text}

{format_instructions}
"""

prompt3 = ChatPromptTemplate.from_template(template=review_template_3)

messages3 = prompt3.format_messages(text=customer_review1, 
                                format_instructions=format_instructions)

In [126]:
print(messages3[0].content)

다음 텍스트의 경우 다음 정보를 추출합니다:

gift: 다른 사람을 위해 선물용으로 구매하셨나요? 예인 경우 True, 아니오 또는 알 수 없는 경우 False로 응답합니다.

delivery_days: 제품이 도착하는 데 며칠이 걸렸나요? 이 정보를 찾을 수 없으면 -1을 출력합니다.

price_value: 값 또는 가격에 대한 모든 문장을 추출하여 쉼표로 구분된 Python 목록으로 출력합니다.

text: 이 나뭇잎 송풍기는 꽤 놀랍습니다. 촛불, 산들바람, 바람이 부는 도시, 토네이도의 네 가지 설정이 있습니다. 아내의 결혼 기념일 선물에 맞춰 이틀 만에 도착했습니다. 아내가 너무 좋아해서 할 말을 잃을 정도였어요. 지금까지는 저 혼자만 사용하고 있으며, 격일로 아침마다 잔디밭의 낙엽을 치우는 데 사용하고 있습니다. 다른 잎사귀 송풍기보다 약간 비싸지만 추가 기능을 고려하면 그만한 가치가 있다고 생각해요.

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "\`\`\`json" and "\`\`\`":

```json
{
	"gift": string  // gift
	"delivery_days": string  // delivery days
	"price_value": string  // price value
}
```



In [127]:
response3 = chat(messages3)

In [128]:
print(response3.content)

```json
{
	"gift": true,
	"delivery_days": 2,
	"price_value": ["약간 비싸지만 추가 기능을 고려하면 그만한 가치가 있다고 생각해요."]
}
```


In [129]:
output_dict3 = output_parser.parse(response3.content)

In [130]:
output_dict3

{'gift': True,
 'delivery_days': 2,
 'price_value': ['약간 비싸지만 추가 기능을 고려하면 그만한 가치가 있다고 생각해요.']}

In [131]:
type(output_dict3)

dict

In [132]:
output_dict3.get('delivery_days')

2